In [65]:
from langchain.agents import create_agent #for creating the agent
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit #for SQL database toolkit
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_core.prompts import ChatPromptTemplate #for creating chat prompts templates
from langchain_google_genai import ChatGoogleGenerativeAI #for using Google Gemini LLM API
from dotenv import load_dotenv
import os

load_dotenv('../credentials.env') #load environment variables (credentials and API keys) from a .env file


True

In [66]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=os.getenv("GOOGLE_API_KEY"),
    
    )

In [68]:
system_prompt = """You are a SQL expert assistant for an expense tracking app. Follow these instructions:
- Always verify table schemas before querying
- Use proper MySQL syntax
- Format results clearly and concisely
- If no data exists, state that explicitly
- Double-check date ranges in queries
- Provide short, actionable insights, tips, or budget suggestions when relevant
- Keep explanations brief and focused on practical guidance"""


In [69]:
mysql_uri = f'mysql+mysqlconnector://{os.getenv("MYSQL_USERNAME")}:{os.getenv("MYSQL_PASSWORD")}@{os.getenv("MYSQL_HOST")}:{os.getenv("MYSQL_PORT")}/{os.getenv("DATABASE_NAME")}'
db = SQLDatabase.from_uri(mysql_uri) # type: ignore

toolkit = SQLDatabaseToolkit(db=db, llm=llm) # setting up the SQL toolkit
agent = create_agent(llm, toolkit.get_tools(), system_prompt=system_prompt) # creating the agent

In [70]:
def invoke_agent(user_query):

    response = agent.invoke({
        "messages": [
            {"role": "user", "content": str(user_query)}
        ]
})
    
    return response


In [ ]:
# response = invoke_agent("Show me the total expenses for each category in the last month, if no data or even a database, just tell me explicity")
# print(response)

In [ ]:
# print(response['messages'][-1].content)

Here are your total expenses per category for the last month:

*   **Shopping:** $45.00
*   **Food:** $12.50
*   **Transport:** $3.00

**Tip:** Review your 'Shopping' expenses. If this is a recurring trend, consider setting a budget for this category to help manage your spending.
